In [2]:
from pathlib import Path
from configparser import ConfigParser


In [3]:
import pandas as pd
import geopandas as gpd
import rasterstats as rs


In [4]:
config_file="config.ini"
config = ConfigParser()
config.read(config_file)

year_list = [int(y) for y in config["main"]["years"].split(", ")]
base_path = Path(config["main"]["output_dir"]) 

start_year = min(year_list)
end_year = max(year_list)

mining_project_id_list = [int(y) for y in config["main"]["mining_project_id_list"].split(", ")]


In [5]:

pre_ndvi_path = base_path / "ndvi" / "output" / "yearly" / "avhrr_ndvi_v5_2001.tif"
post_ndvi_path = base_path / "ndvi" / "output" / "yearly" / "avhrr_ndvi_v5_2011.tif"

pre_pop_path = base_path / "population" / "ppp_2001_1km_Aggregated.tif"
post_pop_path = base_path / "population" / "ppp_2011_1km_Aggregated.tif"

gcdf_path = base_path / "gcdf" / "all_combined_global.gpkg"

In [6]:
gcdf_gdf = gpd.read_file(gcdf_path, driver="GPKG")

/opt/conda/lib/python3.11/site-packages/pyogrio/raw.py:196: RuntimeWarning: driver GPKG does not support open option DRIVER
  return ogr_read(


In [7]:
mining_gdf = gcdf_gdf.loc[gcdf_gdf.id.isin(mining_project_id_list) & (gcdf_gdf["Commitment.Year"] > start_year) & (gcdf_gdf["Completion.Year"] < end_year)].copy()

mining_gdf.geometry = mining_gdf.geometry.buffer(500/111321)

/tmp/ipykernel_1428/2701653876.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  mining_gdf.geometry = mining_gdf.geometry.buffer(500/111321)


In [8]:
len(mining_gdf)

7

In [9]:
pre_ndvi_extract = rs.zonal_stats(mining_gdf.geometry, pre_ndvi_path, stats="mean", geojson_out=True, all_touched=True)
post_ndvi_extract = rs.zonal_stats(mining_gdf.geometry, post_ndvi_path, stats="mean", geojson_out=True, all_touched=True)

pre_ndvi_vals = [(i["id"], i["properties"]["mean"]) for i in pre_ndvi_extract]
post_ndvi_vals = [(i["id"], i["properties"]["mean"]) for i in post_ndvi_extract]

In [14]:
pre_pop_extract = rs.zonal_stats(mining_gdf.geometry, pre_pop_path, stats="sum", geojson_out=True, all_touched=True)
post_pop_extract = rs.zonal_stats(mining_gdf.geometry, post_pop_path, stats="sum", geojson_out=True, all_touched=True)

pre_pop_vals = [(i["id"], i["properties"]["sum"]) for i in pre_pop_extract]
post_pop_vals = [(i["id"], i["properties"]["sum"]) for i in post_pop_extract]

In [15]:
df_list = [
    pd.DataFrame(pre_ndvi_vals, columns=["id", "ndvi_2001"]),
    pd.DataFrame(post_ndvi_vals, columns=["id", "ndvi_2011"]),
    pd.DataFrame(pre_pop_vals, columns=["id", "pop_2001"]),
    pd.DataFrame(post_pop_vals, columns=["id", "pop_2011"]),
]

df = pd.concat(df_list, axis=1)
df = df.loc[:,~df.columns.duplicated()].copy()
df.id = df.id.astype(int)
df = df.set_index("id")


In [16]:

gcdf_fields = ["id", "Recipient", "Recipient.ISO-3", "Title", "Status", "Amount.(Constant.USD.2021)"]
dfx = df.merge(mining_gdf[gcdf_fields], left_index=True, right_index=True)

extract_output_path = base_path / "extract.csv"
dfx.to_csv(extract_output_path, index=False)

In [17]:
dfx

,ndvi_2001,ndvi_2011,pop_2001,pop_2011,id,Recipient,Recipient.ISO-3,Title,Status,Amount.(Constant.USD.2021)
id,,,,,,,,,,
60,1636.750000,1433.937500,12.669615,40.075386,256,Niger,NER,China Eximbank provides RMB 650 million govern...,Completion,1.236623e+08
2005,1196.958333,951.363636,262.151428,1629.226440,39943,Uzbekistan,UZB,ICBC provides $29.2 million export buyer's cre...,Completion,7.123050e+07
2630,4934.200000,4569.111111,1843.439697,2277.384033,52175,Zambia,ZMB,China Development Bank provides $29 million lo...,Completion,3.254599e+07
5141,6164.603535,5377.238095,4420.025391,6291.714355,64488,Viet Nam,VNM,China Eximbank provides RMB 336.15 million gov...,Completion,9.893356e+07
5183,7574.000000,6761.000000,15.062001,22.629385,64615,Ecuador,ECU,"CDB, BoC, and ABC provide $329 million loan to...",Completion,4.582072e+08
5873,1908.125000,1608.409091,13867.176758,11410.609375,67103,Iran,IRN,China Eximbank provides $143.91 million buyer'...,Completion,3.505739e+08
8479,1636.750000,1433.937500,12.669615,40.075386,91977,Niger,NER,China Eximbank provides $142 million buyer’s c...,Completion,2.132854e+08
